<a href="https://colab.research.google.com/github/ralfnovikov1-spec/QEC-Sim-Hardware-Calibrated_Surface_Code_Simulator/blob/main/Uus_quantum_memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
# CELL 2 — Quantum–Classical Bidirectional Coupling
# True backaction, no measurement collapse
# ============================================================

import numpy as np
import qutip as qt

# -----------------------------
# Lorenz-63 parameters
# -----------------------------
sigma = 10.0
rho0  = 28.0
beta  = 8/3

# Coupling strength (tunable)
lambda_q = 5.0   # start moderate

# -----------------------------
# Quantum system
# -----------------------------
sx = qt.sigmax()
sz = qt.sigmaz()
H0 = 0.5 * sx     # base Hamiltonian
gamma = 0.1       # weak decoherence

# -----------------------------
# Lorenz RHS with quantum backaction
# -----------------------------
def lorenz_quantum(state, q_expect):
    x, y, z = state
    rho_eff = rho0 + lambda_q * q_expect
    dx = sigma * (y - x)
    dy = x * (rho_eff - z) - y
    dz = x * y - beta * z
    return np.array([dx, dy, dz])

# -----------------------------
# One RK4 step (classical)
# -----------------------------
def rk4_lorenz_q(x, q_expect, dt):
    k1 = lorenz_quantum(x, q_expect)
    k2 = lorenz_quantum(x + 0.5*dt*k1, q_expect)
    k3 = lorenz_quantum(x + 0.5*dt*k2, q_expect)
    k4 = lorenz_quantum(x + dt*k3, q_expect)
    return x + (dt/6)*(k1 + 2*k2 + 2*k3 + k4)

# -----------------------------
# Quantum evolution step
# -----------------------------
def quantum_step(rho, x_classical, dt):
    H = H0 + 0.5 * x_classical * sz
    c_ops = [np.sqrt(gamma) * sz]
    result = qt.mesolve(H, rho, [0, dt], c_ops, [])
    return result.states[-1]

# -----------------------------
# Coupled trajectory
# -----------------------------
def coupled_trajectory(
    x0,
    dt=0.01,
    T=300.0,
    transient=50.0
):
    steps = int(T/dt)
    transient_steps = int(transient/dt)

    # Initial conditions
    x = np.array(x0, dtype=float)
    rho = qt.basis(2,0) * qt.basis(2,0).dag()

    traj = []

    for i in range(steps):
        # Quantum expectation (NO measurement collapse)
        q_expect = np.real(qt.expect(sz, rho))

        # Classical update
        x = rk4_lorenz_q(x, q_expect, dt)

        # Quantum update (classical feeds back)
        rho = quantum_step(rho, x[0], dt)

        if i > transient_steps:
            traj.append(x.copy())

    return np.array(traj)

# -----------------------------
# Lyapunov exponent (reuse fixed code)
# -----------------------------
def lyapunov_from_trajectory(traj, dt, eps=1e-8):
    x = traj[0]
    delta = np.random.randn(3)
    delta = eps * delta / np.linalg.norm(delta)
    x_pert = x + delta

    lyap_sum = 0.0
    count = 0

    for i in range(1, len(traj)):
        x = traj[i]
        x_pert = traj[i] + (x_pert - x)

        d = np.linalg.norm(x_pert - x)
        if d == 0:
            continue

        lyap_sum += np.log(d / eps)
        count += 1

        x_pert = x + eps * (x_pert - x) / d

    return lyap_sum / (count * dt)

# -----------------------------
# Run experiment
# -----------------------------
np.random.seed(0)
x0 = np.array([1.0, 1.0, 1.0])

traj_q = coupled_trajectory(x0)
lambda_qm = lyapunov_from_trajectory(traj_q, dt=0.01)

print("="*60)
print("QUANTUM–CLASSICAL BACKACTION RESULT")
print("="*60)
print(f"Coupled λ₁ ≈ {lambda_qm:.3f}")
print("Classical λ₁ ≈ 0.90")
print("="*60)

/usr/local/lib/python3.12/dist-packages/qutip/solver/solver_base.py:598: FutureWarning: e_ops will be keyword only from qutip 5.3 for all solver
  warnings.warn(


QUANTUM–CLASSICAL BACKACTION RESULT
Coupled λ₁ ≈ 1823.284
Classical λ₁ ≈ 0.90


In [ ]:
!pip -q install qutip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.6/31.6 MB 85.7 MB/s eta 0:00:00


In [ ]:
# ============================================================
# CELL 1 (FIXED) — Classical Chaos Sanity Check
# Proper Lyapunov Exponent (Benettin Algorithm)
# ============================================================

import numpy as np

# -----------------------------
# Lorenz-63 system
# -----------------------------
def lorenz(state, sigma=10.0, rho=28.0, beta=8/3):
    x, y, z = state
    dx = sigma * (y - x)
    dy = x * (rho - z) - y
    dz = x * y - beta * z
    return np.array([dx, dy, dz])

# -----------------------------
# One integration step (RK4)
# -----------------------------
def rk4_step(f, x, dt, **kwargs):
    k1 = f(x, **kwargs)
    k2 = f(x + 0.5 * dt * k1, **kwargs)
    k3 = f(x + 0.5 * dt * k2, **kwargs)
    k4 = f(x + dt * k3, **kwargs)
    return x + (dt / 6.0) * (k1 + 2*k2 + 2*k3 + k4)

# -----------------------------
# Largest Lyapunov exponent
# Correct Benettin algorithm
# -----------------------------
def lyapunov_exponent(
    f,
    x0,
    dt=0.01,
    T=300.0,
    eps=1e-8,
    transient=50.0,
    **kwargs
):
    steps = int(T / dt)
    transient_steps = int(transient / dt)

    x = np.array(x0, dtype=float)

    # Initialize perturbation with fixed norm eps
    delta = np.random.randn(3)
    delta = eps * delta / np.linalg.norm(delta)
    x_pert = x + delta

    lyap_sum = 0.0
    count = 0

    for i in range(steps):
        x = rk4_step(f, x, dt, **kwargs)
        x_pert = rk4_step(f, x_pert, dt, **kwargs)

        # Distance after evolution
        diff = x_pert - x
        d = np.linalg.norm(diff)

        if d == 0:
            continue

        # Accumulate growth rate
        if i > transient_steps:
            lyap_sum += np.log(d / eps)
            count += 1

        # Renormalize perturbation
        diff = eps * diff / d
        x_pert = x + diff

    return lyap_sum / (count * dt)

# -----------------------------
# Run sanity check
# -----------------------------
np.random.seed(0)
x0 = np.array([1.0, 1.0, 1.0])

lambda1 = lyapunov_exponent(
    lorenz,
    x0,
    dt=0.01,
    T=300.0,
    transient=50.0,
)

print("=" * 60)
print("CLASSICAL LORENZ-63 SANITY CHECK (FIXED)")
print("=" * 60)
print(f"Estimated λ₁ ≈ {lambda1:.3f}")
print("Expected λ₁ ≈ 0.9")
print("=" * 60)

CLASSICAL LORENZ-63 SANITY CHECK (FIXED)
Estimated λ₁ ≈ 0.905
Expected λ₁ ≈ 0.9


In [ ]:
# ============================================================
# CELL 3 — Joint Quantum–Classical Lyapunov Exponent (6D)
# ============================================================

import numpy as np

# -----------------------------
# Parameters
# -----------------------------
sigma = 10.0
rho0  = 28.0
beta  = 8/3

lambda_q = 5.0     # quantum → classical coupling
gamma    = 0.1     # qubit decoherence
omega    = 1.0     # qubit base frequency

dt = 0.01
T  = 300.0
transient = 50.0

# -----------------------------
# Pauli matrices
# -----------------------------
sx = np.array([[0, 1], [1, 0]], dtype=float)
sy = np.array([[0, -1], [1, 0]], dtype=float)
sz = np.array([[1, 0], [0, -1]], dtype=float)

# -----------------------------
# Bloch vector dynamics (Lindblad)
# -----------------------------
def bloch_rhs(r, x_classical):
    rx, ry, rz = r

    # Hamiltonian H = ½(ω σx + x σz)
    hx = omega
    hz = x_classical

    drx = -2*hz*ry - gamma*rx
    dry =  2*hz*rx - 2*hx*rz - gamma*ry
    drz =  2*hx*ry - gamma*(rz + 1)

    return np.array([drx, dry, drz])

# -----------------------------
# Lorenz dynamics with quantum backaction
# -----------------------------
def lorenz_rhs(xyz, rz):
    x, y, z = xyz
    rho_eff = rho0 + lambda_q * rz
    dx = sigma * (y - x)
    dy = x * (rho_eff - z) - y
    dz = x * y - beta * z
    return np.array([dx, dy, dz])

# -----------------------------
# Full 6D system RHS
# -----------------------------
def full_rhs(state):
    x, y, z, rx, ry, rz = state
    dxyz = lorenz_rhs((x, y, z), rz)
    dr   = bloch_rhs((rx, ry, rz), x)
    return np.concatenate([dxyz, dr])

# -----------------------------
# RK4 step
# -----------------------------
def rk4_step(f, s, dt):
    k1 = f(s)
    k2 = f(s + 0.5*dt*k1)
    k3 = f(s + 0.5*dt*k2)
    k4 = f(s + dt*k3)
    return s + (dt/6)*(k1 + 2*k2 + 2*k3 + k4)

# -----------------------------
# Joint Lyapunov exponent (Benettin)
# -----------------------------
def joint_lyapunov(x0, r0, eps=1e-8):
    steps = int(T/dt)
    transient_steps = int(transient/dt)

    s = np.array(list(x0) + list(r0), dtype=float)

    delta = np.random.randn(6)
    delta = eps * delta / np.linalg.norm(delta)
    s_pert = s + delta

    lyap_sum = 0.0
    count = 0

    for i in range(steps):
        s      = rk4_step(full_rhs, s, dt)
        s_pert = rk4_step(full_rhs, s_pert, dt)

        diff = s_pert - s
        d = np.linalg.norm(diff)
        if d == 0:
            continue

        if i > transient_steps:
            lyap_sum += np.log(d / eps)
            count += 1

        s_pert = s + eps * diff / d

    return lyap_sum / (count * dt)

# -----------------------------
# Initial conditions
# -----------------------------
np.random.seed(0)
x0 = np.array([1.0, 1.0, 1.0])
r0 = np.array([0.0, 0.0, -1.0])   # ground state

lambda_joint = joint_lyapunov(x0, r0)

print("="*60)
print("JOINT QUANTUM–CLASSICAL LYAPUNOV RESULT")
print("="*60)
print(f"λ₁ (6D system) ≈ {lambda_joint:.3f}")
print("Classical Lorenz λ₁ ≈ 0.90")
print("="*60)

JOINT QUANTUM–CLASSICAL LYAPUNOV RESULT
λ₁ (6D system) ≈ -0.077
Classical Lorenz λ₁ ≈ 0.90


In [7]:
# ============================================================
# SANITY TEST 1 — Coupling Sweep
# ============================================================

import numpy as np

# -----------------------------
# Parameters (same as CELL 3)
# -----------------------------
sigma = 10.0
rho0  = 28.0
beta  = 8/3

gamma = 0.1
omega = 1.0

dt = 0.01
T  = 300.0
transient = 50.0

# -----------------------------
# Dynamics definitions
# -----------------------------
def bloch_rhs(r, x):
    rx, ry, rz = r
    hx = omega
    hz = x
    drx = -2*hz*ry - gamma*rx
    dry =  2*hz*rx - 2*hx*rz - gamma*ry
    drz =  2*hx*ry - gamma*(rz + 1)
    return np.array([drx, dry, drz])

def lorenz_rhs(xyz, rz, lambda_q):
    x, y, z = xyz
    rho_eff = rho0 + lambda_q * rz
    dx = sigma * (y - x)
    dy = x * (rho_eff - z) - y
    dz = x * y - beta * z
    return np.array([dx, dy, dz])

def full_rhs(state, lambda_q):
    x, y, z, rx, ry, rz = state
    dxyz = lorenz_rhs((x, y, z), rz, lambda_q)
    dr   = bloch_rhs((rx, ry, rz), x)
    return np.concatenate([dxyz, dr])

def rk4_step(state, lambda_q):
    k1 = full_rhs(state, lambda_q)
    k2 = full_rhs(state + 0.5*dt*k1, lambda_q)
    k3 = full_rhs(state + 0.5*dt*k2, lambda_q)
    k4 = full_rhs(state + dt*k3, lambda_q)
    return state + (dt/6)*(k1 + 2*k2 + 2*k3 + k4)

# -----------------------------
# Joint Lyapunov
# -----------------------------
def joint_lyapunov(lambda_q, eps=1e-8):
    steps = int(T/dt)
    transient_steps = int(transient/dt)

    s = np.array([1.0, 1.0, 1.0, 0.0, 0.0, -1.0])
    delta = np.random.randn(6)
    delta = eps * delta / np.linalg.norm(delta)
    s_pert = s + delta

    lyap_sum = 0.0
    count = 0

    for i in range(steps):
        s      = rk4_step(s, lambda_q)
        s_pert = rk4_step(s_pert, lambda_q)

        diff = s_pert - s
        d = np.linalg.norm(diff)
        if d == 0:
            continue

        if i > transient_steps:
            lyap_sum += np.log(d / eps)
            count += 1

        s_pert = s + eps * diff / d

    return lyap_sum / (count * dt)

# -----------------------------
# Run sweep
# -----------------------------
np.random.seed(0)

lambdas = [0.0, 0.5, 1.0, 2.0, 3.0, 5.0, 8.0]
results = {}

print("="*60)
print("COUPLING SWEEP SANITY CHECK")
print("="*60)

for lam in lambdas:
    val = joint_lyapunov(lam)
    results[lam] = val
    print(f"λ_q = {lam:>4}  →  λ₁ ≈ {val:>7.3f}")

print("="*60)

COUPLING SWEEP SANITY CHECK
λ_q =  0.0  →  λ₁ ≈   0.905
λ_q =  0.5  →  λ₁ ≈   0.914
λ_q =  1.0  →  λ₁ ≈   0.907
λ_q =  2.0  →  λ₁ ≈   0.856
λ_q =  3.0  →  λ₁ ≈   0.857
λ_q =  5.0  →  λ₁ ≈  -0.077
λ_q =  8.0  →  λ₁ ≈  -0.066


In [8]:
# ============================================================
# SANITY TEST 2 — Classical Surrogate Control
# ============================================================

import numpy as np

# -----------------------------
# Parameters (same as before)
# -----------------------------
sigma = 10.0
rho0  = 28.0
beta  = 8/3

dt = 0.01
T  = 300.0
transient = 50.0

# Classical surrogate strength
lambda_c = 5.0

# -----------------------------
# Lorenz with classical surrogate
# -----------------------------
def lorenz_surrogate_rhs(xyz):
    x, y, z = xyz
    rz = np.tanh(x)   # bounded classical nonlinearity
    rho_eff = rho0 + lambda_c * rz
    dx = sigma * (y - x)
    dy = x * (rho_eff - z) - y
    dz = x * y - beta * z
    return np.array([dx, dy, dz])

def rk4_step_surrogate(x):
    k1 = lorenz_surrogate_rhs(x)
    k2 = lorenz_surrogate_rhs(x + 0.5*dt*k1)
    k3 = lorenz_surrogate_rhs(x + 0.5*dt*k2)
    k4 = lorenz_surrogate_rhs(x + dt*k3)
    return x + (dt/6)*(k1 + 2*k2 + 2*k3 + k4)

# -----------------------------
# Lyapunov exponent (3D)
# -----------------------------
def lyapunov_surrogate(eps=1e-8):
    steps = int(T/dt)
    transient_steps = int(transient/dt)

    x = np.array([1.0, 1.0, 1.0])
    delta = np.random.randn(3)
    delta = eps * delta / np.linalg.norm(delta)
    x_pert = x + delta

    lyap_sum = 0.0
    count = 0

    for i in range(steps):
        x      = rk4_step_surrogate(x)
        x_pert = rk4_step_surrogate(x_pert)

        diff = x_pert - x
        d = np.linalg.norm(diff)
        if d == 0:
            continue

        if i > transient_steps:
            lyap_sum += np.log(d / eps)
            count += 1

        x_pert = x + eps * diff / d

    return lyap_sum / (count * dt)

# -----------------------------
# Run test
# -----------------------------
np.random.seed(0)

lambda_sur = lyapunov_surrogate()

print("="*60)
print("CLASSICAL SURROGATE SANITY CHECK")
print("="*60)
print(f"λ₁ (classical surrogate) ≈ {lambda_sur:.3f}")
print("Quantum-stabilized λ₁ ≈ -0.07")
print("="*60)

CLASSICAL SURROGATE SANITY CHECK
λ₁ (classical surrogate) ≈ -0.053
Quantum-stabilized λ₁ ≈ -0.07


In [9]:
# ============================================================
# NON-COMMUTING QUANTUM BACKACTION TEST (AUDIT CLEAN)
# σz → ρ , σx → σ
# ============================================================

!pip -q install qutip

import numpy as np
import qutip as qt

# -----------------------------
# Lorenz-63 vector field
# -----------------------------
def lorenz(x, sigma, rho, beta):
    dx = sigma * (x[1] - x[0])
    dy = x[0] * (rho - x[2]) - x[1]
    dz = x[0] * x[1] - beta * x[2]
    return np.array([dx, dy, dz])

# -----------------------------
# Benettin Lyapunov estimator
# -----------------------------
def lyapunov(system_step, x0, dt, T=4000, eps=1e-8):
    x = x0.copy()
    v = np.random.randn(len(x))
    v /= np.linalg.norm(v)

    lyap_sum = 0.0
    for _ in range(T):
        x1 = system_step(x)
        x2 = system_step(x + eps * v)

        dv = x2 - x1
        dist = np.linalg.norm(dv)
        v = dv / dist
        lyap_sum += np.log(dist / eps)
        x = x1

    return lyap_sum / (T * dt)

# -----------------------------
# Quantum system (single qubit)
# -----------------------------
sx, sz = qt.sigmax(), qt.sigmaz()
H0 = 0.3 * sx
c_ops = [np.sqrt(0.1) * sz]

def quantum_step(rho, dt):
    result = qt.mesolve(H0, rho, [0, dt], c_ops=c_ops)
    rho = result.states[-1]
    return rho, np.real(qt.expect(sz, rho)), np.real(qt.expect(sx, rho))

# -----------------------------
# Coupled quantum–classical system
# -----------------------------
def joint_step_factory(lam_q):

    sigma0, rho0, beta = 10.0, 28.0, 8/3
    rho_q = qt.basis(2,0) * qt.basis(2,0).dag()

    def step(x):
        nonlocal rho_q

        # quantum update
        rho_q, ez, ex = quantum_step(rho_q, dt=0.01)

        # non-commuting feedback
        sigma = sigma0 + lam_q * ex
        rho_p = rho0 + lam_q * ez

        return x + lorenz(x, sigma, rho_p, beta) * 0.01

    return step

# -----------------------------
# Classical surrogate (matched statistics)
# -----------------------------
def surrogate_step_factory(lam_q):

    sigma0, rho0, beta = 10.0, 28.0, 8/3
    z_state, x_state = 0.0, 0.0

    def step(x):
        nonlocal z_state, x_state

        # two independent nonlinear channels
        z_state = 0.9 * z_state + 0.1 * np.tanh(x[0])
        x_state = 0.9 * x_state + 0.1 * np.tanh(x[1])

        sigma = sigma0 + lam_q * x_state
        rho_p = rho0 + lam_q * z_state

        return x + lorenz(x, sigma, rho_p, beta) * 0.01

    return step

# ============================================================
# Run sweep
# ============================================================

x0 = np.array([1.0, 1.0, 1.0])
lam_vals = [0.0, 0.5, 1.0, 2.0, 4.0]

print("\n============================================================")
print("NON-COMMUTING BACKACTION SANITY SWEEP")
print("============================================================")

for lam in lam_vals:
    λ_q = lyapunov(joint_step_factory(lam), x0, dt=0.01)
    λ_c = lyapunov(surrogate_step_factory(lam), x0, dt=0.01)

    print(f"λ_q = {lam:>4} → λ₁(quantum) = {λ_q:>7.3f} | λ₁(classical) = {λ_c:>7.3f}")

print("============================================================")


NON-COMMUTING BACKACTION SANITY SWEEP
λ_q =  0.0 → λ₁(quantum) =   0.980 | λ₁(classical) =   0.890
λ_q =  0.5 → λ₁(quantum) = 507.050 | λ₁(classical) = 543.843
λ_q =  1.0 → λ₁(quantum) = 571.306 | λ₁(classical) = 661.743
λ_q =  2.0 → λ₁(quantum) = 637.999 | λ₁(classical) = 762.731
λ_q =  4.0 → λ₁(quantum) = 707.159 | λ₁(classical) = 795.851


In [3]:
# ============================================================
# ENTANGLED QUANTUM BACKREACTION ON CLASSICAL CHAOS
# CORRECT BENETTIN LYAPUNOV IMPLEMENTATION
# ============================================================

!pip -q install qutip numpy scipy

import numpy as np
import qutip as qt

# ============================================================
# 1. Classical Lorenz-63 system
# ============================================================

def lorenz_rhs(state, sigma=10.0, rho=28.0, beta=8/3):
    x, y, z = state
    return np.array([
        sigma * (y - x),
        x * (rho - z) - y,
        x * y - beta * z
    ])

def evolve_lorenz(state, dt, rho_mod):
    k1 = lorenz_rhs(state, rho=rho_mod)
    k2 = lorenz_rhs(state + 0.5*dt*k1, rho=rho_mod)
    k3 = lorenz_rhs(state + 0.5*dt*k2, rho=rho_mod)
    k4 = lorenz_rhs(state + dt*k3, rho=rho_mod)
    return state + (dt/6)*(k1 + 2*k2 + 2*k3 + k4)

# ============================================================
# 2. Correct Lyapunov exponent (Benettin algorithm)
# ============================================================

def lyapunov_benettin(step_fn, x0, dt, T, eps=1e-8):
    x = np.array(x0)
    y = x + eps * np.random.randn(3)
    y = x + eps * (y - x) / np.linalg.norm(y - x)

    s = 0.0
    steps = int(T / dt)

    for _ in range(steps):
        x = step_fn(x)
        y = step_fn(y)

        d = y - x
        dist = np.linalg.norm(d)
        if dist == 0:
            continue

        s += np.log(dist / eps)
        y = x + eps * d / dist

    return s / (steps * dt)

# ============================================================
# 3. Two-qubit quantum system (never measured)
# ============================================================

sz = qt.sigmaz()
sx = qt.sigmax()
I  = qt.qeye(2)

ZZ = qt.tensor(sz, sz)
XX = qt.tensor(sx, sx)

def quantum_expectation(J, operator, T, dt):
    H = (
        J * operator
        + 0.3 * qt.tensor(sz, I)
        + 0.3 * qt.tensor(I, sz)
    )

    psi0 = (
        qt.tensor(qt.basis(2,0), qt.basis(2,0)) +
        qt.tensor(qt.basis(2,1), qt.basis(2,1))
    ).unit()

    rho0 = psi0 * psi0.dag()
    times = np.arange(0, T, dt)

    result = qt.mesolve(H, rho0, times, [], [operator])
    return np.real(result.expect[0])

# ============================================================
# 4. Coupled quantum–classical system
# ============================================================

def coupled_lyapunov(J, operator, lambda_q=1.0,
                     T=60.0, dt=0.01,
                     classical_shadow=False):

    q_signal = quantum_expectation(J, operator, T, dt)

    if classical_shadow:
        rng = np.random.default_rng(0)
        q_signal = rng.normal(
            loc=np.mean(q_signal),
            scale=np.std(q_signal),
            size=len(q_signal)
        )

    idx = 0
    def step(state):
        nonlocal idx
        rho_eff = 28.0 + lambda_q * q_signal[idx]
        idx = (idx + 1) % len(q_signal)
        return evolve_lorenz(state, dt, rho_eff)

    return lyapunov_benettin(
        step_fn=step,
        x0=[1.0, 1.0, 1.0],
        dt=dt,
        T=T
    )

# ============================================================
# 5. FINAL AUDIT SWEEP
# ============================================================

Js = [0.0, 0.5, 1.0, 2.0, 4.0]
lambda_q = 2.0

print("="*60)
print("ENTANGLED QUANTUM BACKREACTION — BENETTIN AUDIT")
print("="*60)

for J in Js:
    lam_q = coupled_lyapunov(J, ZZ, lambda_q=lambda_q, classical_shadow=False)
    lam_c = coupled_lyapunov(J, ZZ, lambda_q=lambda_q, classical_shadow=True)
    print(f"J = {J:>3} | λ₁ quantum = {lam_q:>6.3f} | λ₁ classical shadow = {lam_c:>6.3f}")

print("="*60)

# ============================================================
# 6. Basis sensitivity test
# ============================================================

lam_ZZ = coupled_lyapunov(2.0, ZZ, lambda_q=lambda_q)
lam_XX = coupled_lyapunov(2.0, XX, lambda_q=lambda_q)

print("BASIS TEST")
print(f"λ₁ (ZZ coupling) = {lam_ZZ:.3f}")
print(f"λ₁ (XX coupling) = {lam_XX:.3f}")
print("="*60)

ENTANGLED QUANTUM BACKREACTION — BENETTIN AUDIT
J = 0.0 | λ₁ quantum =  0.760 | λ₁ classical shadow =  0.763
J = 0.5 | λ₁ quantum =  0.762 | λ₁ classical shadow =  0.774
J = 1.0 | λ₁ quantum =  0.775 | λ₁ classical shadow =  0.758
J = 2.0 | λ₁ quantum =  0.758 | λ₁ classical shadow =  0.777
J = 4.0 | λ₁ quantum =  0.719 | λ₁ classical shadow =  0.776
BASIS TEST
λ₁ (ZZ coupling) = 0.774
λ₁ (XX coupling) = 975.065
